In [1]:
from utils.gene_selection import evaluate_gene_selection_method

In [2]:
evaluate_gene_selection_method(dataset='xin', methods=['rf', 'lgb', 'xgb', 'nsc', 'cv2', 'var'], data_type='raw')

6267 genes have been removed.
0 cells have been removed.
The dataset has 1492 cells and 33584 genes.
The method you are using is rf.
MRR:0.03353
marker genes found:106
ari of seurat: 0.38503
ari of sc3: 0.98686
classification report of scmap_cluster:

              precision    recall  f1-score   support

       alpha       1.00      1.00      1.00       275
        beta       1.00      1.00      1.00       135
       delta       1.00      0.92      0.96        13
       gamma       1.00      0.96      0.98        25
  unassigned       0.00      0.00      0.00         0

    accuracy                           0.99       448
   macro avg       0.80      0.78      0.79       448
weighted avg       1.00      0.99      1.00       448

classification report of scmap_cell:

              precision    recall  f1-score   support

       alpha       1.00      1.00      1.00       275
        beta       1.00      1.00      1.00       135
       delta       1.00      0.38      0.56        13
    

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


MRR:0.05413
marker genes found:46
ari of seurat: 0.38306
ari of sc3: 0.99068
classification report of scmap_cluster:

              precision    recall  f1-score   support

       alpha       1.00      0.99      0.99       275
        beta       1.00      1.00      1.00       135
       delta       1.00      0.85      0.92        13
       gamma       1.00      0.96      0.98        25
  unassigned       0.00      0.00      0.00         0

    accuracy                           0.99       448
   macro avg       0.80      0.76      0.78       448
weighted avg       1.00      0.99      0.99       448

classification report of scmap_cell:

              precision    recall  f1-score   support

       alpha       1.00      0.98      0.99       275
        beta       0.99      0.99      0.99       135
       delta       1.00      0.15      0.27        13
       gamma       1.00      0.36      0.53        25
  unassigned       0.00      0.00      0.00         0

    accuracy                 

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


MRR:0.14718
marker genes found:17
ari of seurat: 0.28356
ari of sc3: 0.99450
classification report of scmap_cluster:

              precision    recall  f1-score   support

       alpha       1.00      0.02      0.04       275
        beta       0.00      0.00      0.00       135
       delta       0.00      0.00      0.00        13
       gamma       0.00      0.00      0.00        25
  unassigned       0.00      0.00      0.00         0

    accuracy                           0.01       448
   macro avg       0.20      0.00      0.01       448
weighted avg       0.61      0.01      0.03       448

classification report of scmap_cell:

              precision    recall  f1-score   support

       alpha       0.98      0.69      0.81       275
        beta       1.00      0.56      0.71       135
       delta       0.00      0.00      0.00        13
       gamma       0.00      0.00      0.00        25
  unassigned       0.00      0.00      0.00         0

    accuracy                 

...

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


num of genes whose mean are less than 1e-4:40


/home/tdeng/SingleCell/scRNA-FeatureSelection/utils/evaluate_result.py:19: RuntimeWarning: invalid value encountered in double_scalars
  print('MRR:{:.5f}'.format(np.sum(1 / (rank + 1)) / rank.shape[0]))


MRR:nan
marker genes found:0
ari of seurat: 0.00317


OSError: scRNA-FeatureSelection/tempData/temp_sc3.csv not found.

In [1]:
evaluate_gene_selection_method(dataset='muraro', methods=['cellassign', 'deviance', 'm3drop'], data_type='raw')

229 genes have been removed.
0 cells have been removed.
The method you are using is cellassign.
MRR:0.01396
marker genes found:123
ari of seurat: 0.61380
ari of sc3: 0.92847
classification report of scmap_cluster:

              precision    recall  f1-score   support

      acinar       0.96      1.00      0.98        64
       alpha       1.00      0.98      0.99       246
        beta       0.95      0.96      0.95       126
       delta       1.00      0.98      0.99        62
      ductal       0.94      0.88      0.91        74
 endothelial       1.00      1.00      1.00         5
     epsilon       1.00      1.00      1.00         1
       gamma       1.00      1.00      1.00        31
 mesenchymal       0.96      0.96      0.96        28
  unassigned       0.00      0.00      0.00         0

    accuracy                           0.97       637
   macro avg       0.88      0.88      0.88       637
weighted avg       0.98      0.97      0.97       637

classification report of s

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


MRR:0.03253
marker genes found:121
ari of seurat: 0.60605
ari of sc3: 0.93294
classification report of scmap_cluster:

              precision    recall  f1-score   support

      acinar       0.96      1.00      0.98        64
       alpha       1.00      0.98      0.99       246
        beta       0.95      0.96      0.96       126
       delta       1.00      0.98      0.99        62
      ductal       0.93      0.89      0.91        74
 endothelial       0.71      1.00      0.83         5
     epsilon       1.00      1.00      1.00         1
       gamma       1.00      1.00      1.00        31
 mesenchymal       0.96      0.96      0.96        28
  unassigned       0.00      0.00      0.00         0

    accuracy                           0.97       637
   macro avg       0.85      0.88      0.86       637
weighted avg       0.97      0.97      0.97       637

classification report of scmap_cell:

              precision    recall  f1-score   support

      acinar       0.95      

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


MRR:0.02181
marker genes found:139
ari of seurat: 0.88781
ari of sc3: 0.73237
classification report of scmap_cluster:

              precision    recall  f1-score   support

      acinar       0.96      1.00      0.98        64
       alpha       1.00      0.98      0.99       246
        beta       0.95      0.93      0.94       126
       delta       1.00      0.95      0.98        62
      ductal       0.96      0.88      0.92        74
 endothelial       1.00      1.00      1.00         5
     epsilon       1.00      1.00      1.00         1
       gamma       1.00      0.97      0.98        31
 mesenchymal       1.00      0.96      0.98        28
  unassigned       0.00      0.00      0.00         0

    accuracy                           0.95       637
   macro avg       0.89      0.87      0.88       637
weighted avg       0.98      0.95      0.97       637

classification report of scmap_cell:

              precision    recall  f1-score   support

      acinar       0.95      

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
